This notebook explores possible patterns in ESG data coverage

In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import esg_loader as loader
import qgrid
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from tabulate import tabulate

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

#from jupyter_datatables import init_datatables_mode
# init_datatables_mode()

In [3]:
# load files
meta = loader.load_metadata('../data/esg_metadata.csv', '../data/ESG_wdi.feather')
data = pd.read_feather('../data/ESG_wdi.feather')

# convert date to int and drop everything before 2000 and after 2019
data = data[(data.date>=2000)&(data.date<=2019)]
data['date'] = data['date'].apply(lambda x: int(x))

In [4]:
# Calculate number of countries by indicator and year
# data.info()
counts = data.pivot_table(index='indicatorID', columns='date', values='value', aggfunc='count')
counts.head(10)

date,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
indicatorID,,,,,,,,,,,,,,,,,,,,
AG.LND.AGRI.ZS,206.0,206.0,206.0,206.0,206.0,206.0,208.0,209.0,208.0,208.0,208.0,208.0,208.0,208.0,208.0,208.0,208.0,NaN,NaN,NaN
AG.LND.FRST.ZS,209.0,209.0,209.0,209.0,209.0,209.0,209.0,209.0,209.0,209.0,209.0,209.0,209.0,209.0,209.0,209.0,208.0,NaN,NaN,NaN
AG.LND.TOTL.K2,215.0,215.0,215.0,215.0,215.0,215.0,215.0,215.0,215.0,215.0,215.0,215.0,215.0,215.0,215.0,215.0,215.0,215.0,210.0,NaN
AG.PRD.FOOD.XD,200.0,200.0,200.0,200.0,200.0,200.0,202.0,202.0,202.0,202.0,202.0,202.0,201.0,201.0,201.0,201.0,201.0,NaN,NaN,NaN
BN.CAB.XOKA.GD.ZS,146.0,149.0,153.0,155.0,158.0,170.0,173.0,176.0,177.0,179.0,180.0,182.0,182.0,182.0,181.0,179.0,179.0,175.0,160.0,NaN
CC.EST,194.0,NaN,195.0,195.0,199.0,199.0,199.0,200.0,201.0,202.0,203.0,203.0,203.0,203.0,203.0,203.0,203.0,203.0,203.0,NaN
DT.ODA.ODAT.CD,144.0,144.0,144.0,143.0,143.0,145.0,145.0,145.0,143.0,144.0,144.0,141.0,141.0,141.0,140.0,140.0,140.0,140.0,NaN,NaN
EG.CFT.ACCS.ZS,189.0,189.0,189.0,189.0,189.0,189.0,189.0,189.0,189.0,189.0,189.0,189.0,189.0,189.0,189.0,189.0,189.0,NaN,NaN,NaN
EG.EGY.PRIM.PP.KD,190.0,190.0,191.0,191.0,191.0,192.0,192.0,192.0,192.0,192.0,192.0,192.0,193.0,193.0,192.0,191.0,NaN,NaN,NaN,NaN


In [5]:
# calculate MRV year for each indicator
mrv = data.groupby('indicatorID').max()['date'].rename('mrv')

df = counts.agg(['count', 'mean', 'std', 'min', 'max'], axis=1).sort_values('mean', ascending=False)
df = df.join(mrv)

qgrid.show_grid(df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

The table above provides a lot of insight. Look for indicators with:

* low (year) count: possibly low or intermittent temporal coverage
* low max or mean: sparse country coverage
* large differences between min and max indicate major changes in country coverage over time
* 0 or very low std suggests steady country coverage over time, while high std indicates variable country coverage - possibly originating from household survey
* low MRV may indicate discontinued or structural lag


In [6]:
# find the year corresponding to the maximum drop in country coverage for each indicator
# this helps identify indicators that suddenly stop or drop quickly in terms of country
# coverage

i = (counts.fillna(0).diff(axis=1).idxmin(axis=1) - 1)\
        .sort_values(ascending=False).rename('Maximum Year')
    
qgrid.show_grid(i)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [7]:
# Let's look to see if any indicators show improvement over time.
# here are linear regression results for each indicator
regress_data = counts.fillna(axis=1, method='ffill').fillna(axis=1,method='bfill')

slopes = regress_data.apply(lambda x: stats.linregress(counts.columns, x)[0], axis=1)
pval   = regress_data.apply(lambda x: stats.linregress(counts.columns, x)[3], axis=1)

results = pd.DataFrame({'slope': slopes, 'pval': pval})
results[results.slope>1].sort_values('slope', ascending=False)

,slope,pval
indicatorID,,
IT.NET.BBND.P2,5.567669,2.077868e-04
ENF.CONT.COEN.COST.ZS,3.578947,6.508352e-11
IC.REG.DURS,3.430827,4.225899e-10
IC.TAX.TOTL.CP.ZS,1.821805,7.258145e-11
SE.ADT.LITR.ZS,1.800000,1.509227e-03
IP.PAT.NRES,1.781955,2.110503e-07
EG.ELC.ACCS.ZS,1.731579,1.398700e-07
IP.PAT.RESD,1.719549,1.272189e-08
GFDD.DM.06,1.717293,7.000750e-14


In [8]:
# interactive tool for looking at specific indicator coverages.
# IMO: SE.ADT.LITR.ZS and SI.SPR.PCAP.ZG are dubious as to whether they represent
# "steady improvement"

dropdown = list(df.index)
dropdown.sort()

@interact
def indicator_drilldown(limit=dropdown):
    plt.figure(figsize=(10,5))
    counts.loc[limit].plot(kind='bar')

    plt.figure(figsize=(10,10))
    df = data.copy(deep=True)
    df.loc[df.indicatorID!=limit, 'value'] = np.nan
    sns.heatmap(df.pivot_table(index='iso3c', columns='date', values='value', aggfunc='count'), cmap='Blues')

interactive(children=(Dropdown(description='limit', options=('AG.LND.AGRI.ZS', 'AG.LND.FRST.ZS', 'AG.LND.TOTL.…

In [9]:
pd.options.display.max_colwidth = 100

lowc = counts.max(axis=1).rename('max_count')

meta.set_index('cetsid')[['input_name']].join(lowc[lowc<100].astype('int64'), how='right')


,input_name,max_count
indicatorID,,
EN.CLC.GHGR.MT.CE,GHG net emissions/removals by LUCF (Mt of CO2 equivalent),58
GB.XPD.RSDV.GD.ZS,Research and development expenditure (% of GDP),99
IC.FRM.OUTG.ZS,Value lost due to electrical outages (% of sales for affected firms),51
SE.ADT.LITR.ZS,"Literacy rate, adult total (% of people ages 15 and above)",71
SH.MLR.INCD.P3,"Incidence of malaria (per 1,000 population at risk)",99
SH.STA.SMSS.ZS,People using safely managed sanitation services (% of population),94
SI.DST.FRST.20,Income share held by lowest 20%,84
SI.POV.DDAY,Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population),84
SI.POV.GINI,GINI index (World Bank estimate),84
